<a href="https://colab.research.google.com/github/manishiitg/ML_Experiments/blob/master/nlp/pytorch_improved_lstm_bidirectional_and_2_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

using spacy for token

include_lengths=True. this basically sets set length of the string as well when we fetch the batch data. Hence it will return tuple of actual vector / length.

this is mainly done so that in our NN module we have length's and we can pad/pack the sequence 

most code taken from https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb




In [0]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.3MB/s]


In [3]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))


print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

print(vars(train_data.examples[0]))

train_data, valid_data = train_data.split(random_state = random.seed(SEED))


print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of testing examples: 25000
{'text': ['I', 'very', 'nearly', 'walked', 'out', ',', 'but', 'I', "'d", 'paid', 'my', 'money', ',', 'and', 'my', 'nearly', '-', 'as', '-', 'disgusted', 'friend', 'wanted', 'to', 'hold', 'out', '.', 'After', 'the', 'endearing', ',', 'wide', '-', 'eyed', 'innocence', 'of', '"', 'A', 'New', 'Hope', '"', 'and', 'the', 'thrilling', 'sophistication', 'of', '"', 'The', 'Empire', 'Strikes', 'Back', ',', '"', 'I', 'remember', 'awaiting', '"', 'Return', 'of', 'the', 'Jedi', '"', 'with', 'almost', 'aching', 'anticipation', '.', 'But', 'from', 'the', 'opening', 'scene', 'of', 'this', 'insultingly', 'commercial', 'sewage', ',', 'I', 'was', 'bitterly', 'disappointed', ',', 'and', 'enraged', 'at', 'Lucas', '.', 'He', 'should', 'have', 'been', 'ashamed', 'of', 'himself', ',', 'but', 'this', 'abomination', 'undeniably', 'proves', 'that', 'he', 'does', "n't", 'have', 'subatomic', 'particle', 'of', 'shame', 'in', 'his', 'cold', ',', 'g

In [4]:
from torchtext.vocab import Vectors, GloVe


TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:31, 2.20MB/s]                           
100%|█████████▉| 399904/400000 [00:45<00:00, 8602.59it/s]

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx, weights):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        self.embedding.weight = nn.Parameter(weights, requires_grad=False)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [7]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]



model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX,
            TEXT.vocab.vectors)

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

torch.Size([84900, 300])


In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,720,257 trainable parameters


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)


def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [10]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        text, text_lengths = batch.text
        
        text = text.to(device)
        text_lengths = text_lengths.to(device)
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

train(model, train_iterator, optimizer, criterion)

100%|█████████▉| 399904/400000 [01:00<00:00, 8602.59it/s]

(0.6341418121010065, 0.6328375401596228)

In [11]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

evaluate(model, valid_iterator, criterion)

(0.6246603389820421, 0.6869352409638554)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [13]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 28s
	Train Loss: 0.595 | Train Acc: 68.77%
	 Val. Loss: 0.483 |  Val. Acc: 77.80%
Epoch: 02 | Epoch Time: 0m 28s
	Train Loss: 0.532 | Train Acc: 73.13%
	 Val. Loss: 0.480 |  Val. Acc: 76.90%
Epoch: 03 | Epoch Time: 0m 28s
	Train Loss: 0.549 | Train Acc: 73.10%
	 Val. Loss: 0.507 |  Val. Acc: 75.02%
Epoch: 04 | Epoch Time: 0m 28s
	Train Loss: 0.585 | Train Acc: 68.90%
	 Val. Loss: 0.583 |  Val. Acc: 70.05%
Epoch: 05 | Epoch Time: 0m 28s
	Train Loss: 0.469 | Train Acc: 78.26%
	 Val. Loss: 0.359 |  Val. Acc: 85.35%


better accuracy 